In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools 
import multiprocessing as mp
import numpy as np
import sys

sys.path.append('../')
from baseModels.SVM.model import SVM

In [2]:
feature_dir = '../featEngg/ifeatMethods/data/featvec/'
feature_files = [f.name for f in os.scandir(feature_dir) if f.name.endswith('.csv.gz')]

In [3]:
label_file = '../data/LabelFiles/EnzymeLabelsMultiClass.csv'

In [4]:
def model_evaluate(filename,random_seed):
    df1 = pd.read_csv(feature_dir+filename,header=None)
    df2 = pd.read_csv(label_file,header=None)
    df = df1.merge(df2,on=0)
    enz_names = df[0].values
    X = df.iloc[:,1:-1].values
    y = df.iloc[:,-1].values
    if X.shape[1]<55:
        n_comp = int(0.75*X.shape[1])
    else:
        n_comp=55
    X_train, X_test, y_train, y_test,enz_train,enz_test = train_test_split(X, y,enz_names, test_size=0.25, random_state=random_seed)
    svm = SVM(X_train,X_test,y_train,y_test,verbose=False,optimize=False, pca_comp=n_comp,kern='rbf',regC=30)
    return svm.acc_train,svm.acc_test
 

In [5]:
mp.cpu_count()

24

In [6]:
pool = mp.Pool(mp.cpu_count())

In [7]:
iterable = list(itertools.product(feature_files,range(1000)))

In [8]:
%%time
model_accs = list(pool.starmap(model_evaluate,iterable))

CPU times: user 224 ms, sys: 123 ms, total: 347 ms
Wall time: 3min 38s


In [9]:
mydict = dict()

for m,acc in zip(iterable,model_accs):
    model_name=m[0].replace('.csv.gz','')
    seed = m[1]
    if model_name not in mydict:
        mydict[model_name] = {seed:acc}
    else:
        mydict[model_name][seed] = acc

In [10]:
def get_model_stats(mname):
    train_accs = [v[0] for k,v in mydict[mname].items()]
    test_accs = [v[1] for k,v in mydict[mname].items()]
    return {'train_min':min(train_accs),'train_mean':np.mean(train_accs),
            'train_max':max(train_accs),'train_std':np.std(train_accs),
            'test_min':min(test_accs),'test_mean':np.mean(test_accs),
            'test_max':max(test_accs),'test_std':np.std(test_accs)}

In [11]:
get_model_stats('Geary')

{'train_min': 0.8837209302325582,
 'train_mean': 0.9325116279069768,
 'train_max': 0.9883720930232558,
 'train_std': 0.01649189780915939,
 'test_min': 0.5172413793103449,
 'test_mean': 0.7660689655172414,
 'test_max': 0.9655172413793104,
 'test_std': 0.07776105967579464}

In [12]:
results_dict= {mname:get_model_stats(mname) for mname in mydict.keys()}
    

In [13]:
results_df = pd.DataFrame(results_dict).T

In [14]:
results_df.to_csv('../data/SimResults/iFeatResults.csv',index=True)

In [15]:
results_df.sort_values('train_mean')

,train_min,train_mean,train_max,train_std,test_min,test_mean,test_max,test_std
GAAC,0.627907,0.706988,0.813953,0.029731,0.275862,0.527793,0.758621,0.081180
SOCNumber,0.825581,0.890767,0.965116,0.024321,0.448276,0.695517,0.931034,0.078535
CTDD,0.860465,0.906453,0.965116,0.019004,0.413793,0.648103,0.896552,0.078595
PAAC,0.848837,0.912081,0.976744,0.018937,0.379310,0.656310,0.896552,0.082830
AAC,0.860465,0.914884,0.965116,0.019133,0.379310,0.659034,0.896552,0.080703
APAAC,0.848837,0.916802,0.965116,0.018506,0.413793,0.675345,0.896552,0.081168
CTDC,0.837209,0.917209,0.976744,0.020112,0.413793,0.683448,0.965517,0.086631
NMBroto,0.872093,0.922581,0.988372,0.018137,0.482759,0.758069,1.000000,0.077779
QSOrder,0.883721,0.924791,0.965116,0.016830,0.413793,0.696103,0.931034,0.080250
CTDT,0.872093,0.927988,0.988372,0.017824,0.413793,0.666172,0.862069,0.078370
